In [30]:
import numpy as np
NORMAL = "normal"
MDD = "mdd"
MIXED_BIPOLAR = "mixed bipolar"
PURE_MANIA = "pure mania"

In [31]:
from scipy.stats import multinomial
from typing import List

def markov_sequence(p_init: np.array, p_transition: np.array, sequence_length: int) -> List[int]:
    """
    Generate a Markov sequence based on p_init and p_transition.
    """
    initial_state = list(multinomial.rvs(1, p_init)).index(1)

    states = [initial_state]
    for _ in range(sequence_length - 1):
        p_tr = p_transition[states[-1]]
        new_state = list(multinomial.rvs(1, p_tr)).index(1)
        states.append(new_state)
    return states

In [32]:
p_init = np.array([1.0, 0.0, 0.0])
p_transition = np.array(
    [[0.99610, 0.00382, 0.00008],
     [0.4055, 0.5802, 0.0143],
     [0.999926, 0.000034, 0.00004]]
)

In [33]:
state_sequence = markov_sequence(p_init, p_transition, sequence_length=52)

def eval_sequence(state_sequence):
    """
    Evaluate a state sequence and return the diagnosis.
    """
    NORMAL = 0
    MANIC = 1
    DEPRESSED = 2
    arr = np.array(state_sequence)
    cont_depressed = False
    manic_trans = False

    curr = 0
    for i in range(len(arr)):
        if arr[i] == DEPRESSED:
            curr += 1
        elif arr[i] == MANIC:
            manic_trans = True
            curr = 0
        else:
            curr = 0
        if curr >= 2:
            cont_depressed = True
    return cont_depressed, manic_trans

def diagnosis(cont_depressed, manic_trans):
    if cont_depressed and manic_trans:
        return MIXED_BIPOLAR
    elif cont_depressed:
        return MDD
    elif manic_trans:
        return PURE_MANIA
    else:
        return NORMAL




In [34]:
def simulate_population(p_init, p_transition, sequence_length=52, population_size = 1000):
    population = {NORMAL : 0, MDD : 0, PURE_MANIA: 0, MIXED_BIPOLAR : 0}
    for sim in range(population_size):
        state_sequence = markov_sequence(p_init, p_transition, sequence_length)
        diag = diagnosis(*eval_sequence(state_sequence))
        population[diag] += 1
    return population


def output_probabilities(population):
    population_size = sum(population.values())
    for k, v in population.items():
        print(f"{k}: {v/population_size}")

output_probabilities(simulate_population(p_init, p_transition))

normal: 0.835
mdd: 0.0
pure mania: 0.165
mixed bipolar: 0.0
